In [1]:
import requests as rq
import pymongo as pm
import ujson as js
import pandas as pd
from datetime import datetime as dt
from multiprocessing.dummy import Pool

from ApiHandler import ApiHandler

In [10]:
class ApiRegionOrders(ApiHandler):
    settings = {
        **ApiHandler.settings,
    }
    url = {
        **ApiHandler.url,
        'data': 'https://esi.evetech.net/latest/markets/{region_id}/orders/'
    }
    sql = {
        'insert': 'INSERT INTO {table} ({cols}) VALUES ({vals}) ON DUPLICATE KEY UPDATE {upsert};',
        'delete': """\
        DELETE FROM MarketOrders
        WHERE system_id IN (
                SELECT Sys.system_id
                FROM MapSystems AS Sys
                JOIN MapConstellations AS Const
                    ON Sys.constellation_id = Const.constellation_id
                WHERE Const.region_id = {region_id}
            )
            AND record_time != "{time}"
        ;"""
    }
    script_vals = {
        **ApiHandler.script_vals,
        'table': 'MarketOrders',
    }
    name = 'ESI Region Orders API'
    params = {'order_type': 'all'}
    auth = False
    delete = True
    
    def __init__(self, region_id, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.url_params['region_id'] = region_id
        self.run_process()
    
    def build_data(self, raw_data):
        if type(raw_data) is list:
            data_frame = pd.concat([pd.DataFrame(js.loads(raw)) for raw in raw_data], axis=0)
        else:
            data_frame = pd.DataFrame(js.loads(raw_data))
        data_frame['record_time'] = self.timestamp
        data_frame['issued'] = data_frame['issued'].apply(lambda x: dt.strptime(x, '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d %H:%M:%S'))
        data_frame.rename(columns={'range': 'order_range'}, inplace=True)
        return data_frame
    
    def data_inserter(self, data_frame):
        if self.verbose: self._verbose('insert_data', 'Inserting records...')
        
        insert_script = self.sql['insert'].format(**{
            'table': self.script_vals['table'],
            'cols': ','.join(data_frame.columns),
            'vals': ','.join(['%s']*data_frame.columns.size),
            'upsert': ','.join(['{col}=VALUES({col})'.format(col=col) for col in data_frame.columns])
        })
        
        cur = self.conn['maria'].cursor()
        
        for row in data_frame.itertuples():
            cur.execute(
                insert_script,
                [val if pd.notnull(val) else None for val in row[1:]]
            )
        self.conn['maria'].commit()
        
        if self.delete:
            if self.verbose: self._verbose('insert_data', 'Deleting old records...')
            delete_script = self.sql['delete'].format(**{
                'table': self.script_vals['table'],
                'region_id': self.url_params['region_id'],
                'time': dt.strptime(self.timestamp, '%Y-%m-%d %H:%M:%S')
            })
            cur.execute(delete_script)
            self.conn['maria'].commit()
        
        cur.close()
        
        if self.verbose: self._verbose('insert_data', 'Records inserted.')

In [11]:
pool = Pool(processes=2)
[
    pool.apply(ApiRegionOrders, args=(region,))
    for region in (10000043, 10000052)
]